#### Lab 7 
#### Rouge Metrics

In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import numpy as np
from fuzzywuzzy import fuzz 
import ipywidgets as widgets
import pprint
from ipywidgets import interact, interact_manual
import re
__PATH__ = "./data.csv"

/home/sultan/miniconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


/opt/tljh/user/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
#import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/jupyter-
[nltk_data]     alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter-
[nltk_data]     alex/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
df = pd.read_csv(__PATH__,sep=";",header=0)

In [3]:
df

,id,updatedDate,publishedDate,title,summary,authors,category,metaData,downloadLink,filePath
0,http://arxiv.org/abs/1407.6950v1,2014-07-24T16:56:39Z,2014-07-24T16:56:39Z,"How,whenAndHowMuchACardDeckIsWellShuffled.pdf",The Thesis Consider The Mixing Of Few 3 4 ...,Benjamin Isac Fargion,cs.DM,"Italian Thesis In Engeenering Computer, 26 Feb...",http://arxiv.org/pdf/1407.6950v1.pdf,"./files/How,whenAndHowMuchACardDeckIsWellShuff..."
1,http://arxiv.org/abs/0907.0618v1,2009-07-03T12:35:10Z,2009-07-03T12:35:10Z,QuantumIsometryGroups.pdf,This Thesis Contains The Formulation And Com...,Jyotishman Bhowmick,math.OA,Thesis,http://arxiv.org/pdf/0907.0618v1.pdf,./files/QuantumIsometryGroups.pdf
2,http://arxiv.org/abs/1806.09601v2,2018-07-14T17:06:27Z,2018-06-25T17:55:59Z,ComputationAndBoundingOfFolkmanNumbers.pdf,Phd Thesis Under The Supervision Of Professo...,Aleksandar Bikov,math.CO,PhD Thesis,http://arxiv.org/pdf/1806.09601v2.pdf,./files/ComputationAndBoundingOfFolkmanNumbers...
3,http://arxiv.org/abs/1905.03014v1,2019-05-08T11:47:34Z,2019-05-08T11:47:34Z,OnChurch'sThesisInCubicalAssemblies.pdf,"We Show That Church's Thesis, The Axiom Stat...","Andrew Swan, Taichi Uemura,",math.LO,0,http://arxiv.org/pdf/1905.03014v1.pdf,./files/OnChurch'sThesisInCubicalAssemblies.pdf
4,http://arxiv.org/abs/1901.04911v1,2019-01-15T16:24:07Z,2019-01-15T16:24:07Z,UnconstrainedChurchTuringThesisCannotPossiblyB...,The Church Turing Thesis Asserts That If A P...,Yuri Gurevich,cs.LO,0,http://arxiv.org/pdf/1901.04911v1.pdf,./files/UnconstrainedChurchTuringThesisCannotP...
...,...,...,...,...,...,...,...,...,...,...
991,http://arxiv.org/abs/0709.0497v1,2007-09-04T18:40:50Z,2007-09-04T18:40:50Z,HardSpectatorInteractionsInBToPiPiAtOrderAlpha...,In The Present Thesis I Discuss The Hard Spe...,Volker Pilipp,hep-ph,PhD thesis,http://arxiv.org/pdf/0709.0497v1.pdf,./files/HardSpectatorInteractionsInBToPiPiAtOr...
992,http://arxiv.org/abs/0709.2936v1,2007-09-18T23:56:17Z,2007-09-18T23:56:17Z,BayesianClassificationAndRegressionWithHighDim...,This Thesis Responds To The Challenges Of Us...,Longhai Li,stat.ML,"PhD Thesis Submitted to University of Toronto,...",http://arxiv.org/pdf/0709.2936v1.pdf,./files/BayesianClassificationAndRegressionWit...
993,http://arxiv.org/abs/0710.1790v1,2007-10-09T15:20:08Z,2007-10-09T15:20:08Z,StaticAndDynamicPropertiesOfHadronicSystemsWit...,This Thesis Has Been Devoted To The Study Of...,J. M. Verde-Velasco,hep-ph,"PhD thesis, Supervisors: E. Hernandez, J. Niev...",http://arxiv.org/pdf/0710.1790v1.pdf,./files/StaticAndDynamicPropertiesOfHadronicSy...
994,http://arxiv.org/abs/0711.0873v2,2007-11-11T19:21:04Z,2007-11-06T14:13:50Z,AConformalApproachToNumericalCalculationsOfAsy...,This Thesis Is Concerned With The Developmen...,Anıl Zenginoğlu,gr-qc,"PhD thesis, Max-Planck Institute for Gravitati...",http://arxiv.org/pdf/0711.0873v2.pdf,./files/AConformalApproachToNumericalCalculati...


In [8]:
df.shape

(996, 10)

#### Preprocessing the title to list of tokens

In [4]:
titles = list(df['title'].apply(
    lambda t : 
        tuple(
            filter(lambda e:not e in stopwords.words('english'),
                map(lambda e:e.lower(),
                       re.findall('([A-Z]{1}[a-z]+)',t.replace('.pdf','')))
                )
            )
        )
    )

In [46]:
titles[:20]

[('much', 'card', 'deck', 'well', 'shuffled'),
 ('quantum', 'isometry', 'groups'),
 ('computation', 'bounding', 'folkman', 'numbers'),
 ('church', 'thesis', 'cubical', 'assemblies'),
 ('unconstrained', 'church', 'turing', 'thesis', 'cannot', 'possibly', 'true'),
 ('algebraic',
  'relaxations',
  'hardness',
  'results',
  'polynomial',
  'optimization',
  'lyapunov',
  'analysis'),
 ('resolving',
  'complexity',
  'fundamental',
  'problems',
  'computational',
  'social',
  'choice'),
 ('pa',
  'instantiationally',
  'complete',
  'algorithmically',
  'incomplete',
  'alternative',
  'interpretation',
  'goedelian',
  'incompleteness',
  'church',
  'thesis',
  'links',
  'formal',
  'logic',
  'computability'),
 ('numerical',
  'modeling',
  'fluid',
  'flow',
  'porous',
  'media',
  'modelowanie',
  'numeryczne',
  'transportu',
  'plynow',
  'przez',
  'osrodki',
  'porowate'),
 ('threebranes', 'theory'),
 ('spin',
  'torque',
  'nano',
  'oscillators',
  'memristors',
  'multi',


In [47]:
res = {}
for title in titles[:20]:
    synsets = {}
    for word in title:
        synsets[word]=[synset for synset in wn.synsets(word)]
    res[title] = synsets

In [48]:
res

{('much',
  'card',
  'deck',
  'well',
  'shuffled'): {'much': [Synset('much.n.01'),
   Synset('much.a.01'),
   Synset('much.r.01'),
   Synset('much.r.02'),
   Synset('a_lot.r.01'),
   Synset('much.r.04'),
   Synset('much.r.05')], 'card': [Synset('card.n.01'),
   Synset('card.n.02'),
   Synset('card.n.03'),
   Synset('card.n.04'),
   Synset('wag.n.01'),
   Synset('poster.n.01'),
   Synset('calling_card.n.02'),
   Synset('card.n.08'),
   Synset('menu.n.01'),
   Synset('batting_order.n.01'),
   Synset('circuit_board.n.01'),
   Synset('tease.v.07'),
   Synset('card.v.02')], 'deck': [Synset('deck.n.01'),
   Synset('deck.n.02'),
   Synset('pack_of_cards.n.01'),
   Synset('deck.n.04'),
   Synset('deck.v.01'),
   Synset('deck.v.02'),
   Synset('deck.v.03')], 'well': [Synset('well.n.01'),
   Synset('well.n.02'),
   Synset('well.n.03'),
   Synset('well.n.04'),
   Synset('well.n.05'),
   Synset('well.v.01'),
   Synset('well.a.01'),
   Synset('good.s.13'),
   Synset('well.s.03'),
   Synset('well

In [49]:
reslist = list(res.items())
print(len(reslist))
print(reslist[0])

20
(('much', 'card', 'deck', 'well', 'shuffled'), {'much': [Synset('much.n.01'), Synset('much.a.01'), Synset('much.r.01'), Synset('much.r.02'), Synset('a_lot.r.01'), Synset('much.r.04'), Synset('much.r.05')], 'card': [Synset('card.n.01'), Synset('card.n.02'), Synset('card.n.03'), Synset('card.n.04'), Synset('wag.n.01'), Synset('poster.n.01'), Synset('calling_card.n.02'), Synset('card.n.08'), Synset('menu.n.01'), Synset('batting_order.n.01'), Synset('circuit_board.n.01'), Synset('tease.v.07'), Synset('card.v.02')], 'deck': [Synset('deck.n.01'), Synset('deck.n.02'), Synset('pack_of_cards.n.01'), Synset('deck.n.04'), Synset('deck.v.01'), Synset('deck.v.02'), Synset('deck.v.03')], 'well': [Synset('well.n.01'), Synset('well.n.02'), Synset('well.n.03'), Synset('well.n.04'), Synset('well.n.05'), Synset('well.v.01'), Synset('well.a.01'), Synset('good.s.13'), Synset('well.s.03'), Synset('well.r.01'), Synset('well.r.02'), Synset('well.r.03'), Synset('well.r.04'), Synset('well.r.05'), Synset('wel

In [50]:
print(reslist[0][1]['much'])

[Synset('much.n.01'), Synset('much.a.01'), Synset('much.r.01'), Synset('much.r.02'), Synset('a_lot.r.01'), Synset('much.r.04'), Synset('much.r.05')]


In [51]:
print(reslist[0][0])

('much', 'card', 'deck', 'well', 'shuffled')


In [52]:
resset = set(reslist[0][0])
resset

{'card', 'deck', 'much', 'shuffled', 'well'}

In [64]:
for word in resset:
    print(type(word), word)

<class 'str'> shuffled
<class 'str'> much
<class 'str'> well
<class 'str'> card
<class 'str'> deck


In [54]:
shufsynset = wn.synsets('shuffled')
shufsynset

[Synset('shuffle.v.01'), Synset('shuffle.v.02'), Synset('shuffle.v.03')]

In [99]:
synset = wn.synsets('shuffled')[0]
synset.hypernyms()

[Synset('walk.v.01')]

In [40]:
def get_parent_classes(synset):
    hyps = set()
    while True:
        try:
            synset = synset.hypernyms()[-1]
            hyps.add(synset)
        except IndexError:
            break
    return hyps

In [55]:
get_parent_classes(shufsynset[0])

{Synset('travel.v.01'), Synset('walk.v.01')}

In [91]:
def extract_hypornyms_en(a):
    hypornyms_en = set()
    for word in a:
        # find the hypernyms of the word
        # word = wn.synset(word)
        try:
            word_synset0 = wn.synsets(word)[0]
            hyps_buff = get_parent_classes(word_synset0)
            for h in hyps_buff:
                hypornyms_en.add(h)
        except IndexError:
            continue
    return hypornyms_en

In [92]:
extract_hypornyms_en(resset)

{Synset('abstraction.n.06'),
 Synset('artifact.n.01'),
 Synset('entity.n.01'),
 Synset('excavation.n.03'),
 Synset('horizontal_surface.n.01'),
 Synset('indefinite_quantity.n.01'),
 Synset('large_indefinite_quantity.n.01'),
 Synset('material.n.01'),
 Synset('measure.n.02'),
 Synset('object.n.01'),
 Synset('paper.n.01'),
 Synset('part.n.01'),
 Synset('physical_entity.n.01'),
 Synset('platform.n.01'),
 Synset('relation.n.01'),
 Synset('substance.n.01'),
 Synset('surface.n.01'),
 Synset('travel.v.01'),
 Synset('walk.v.01'),
 Synset('whole.n.02')}

In [93]:
def f1measure(a, b):
    a = set(a)
    b = set(b)
    # missed part
    overlap = set()
    overlap_hyp_cnt = 0
    overlap = a.intersection(b)
    overlap_hyp_cnt = len(overlap) 
    
    recl = overlap_hyp_cnt/len(a)
    prec = overlap_hyp_cnt/len(b)
    
    if len(overlap) == 0:
        return 0, overlap
    else:
        return 2*recl*prec/(recl+prec), overlap

In [70]:
print(reslist[0][0])
print(reslist[1][0])

('much', 'card', 'deck', 'well', 'shuffled')
('quantum', 'isometry', 'groups')


In [84]:
aset = set(reslist[0][0])
bset = set(('much', 'card', 'deck'))

In [85]:
print(aset.intersection(bset))

{'deck', 'much', 'card'}


In [78]:
sss = set(reslist[0][1])
sss

{'card', 'deck', 'much', 'shuffled', 'well'}

In [89]:
f1measure(reslist[0][0], reslist[1][0])

(0, set())

In [94]:
lang = 'eng'

    
#def distance(a,b):
    ### Put your code here
    ### В переменной synsets помимо нормализованного заголовка хранятся синсеты для каждого токена из заголовка.
    ### у синсетов есть гиперонимы про то как из брать здесь
    ### Если у токенов заголовков есть общие гиперонимы тогда заголовки связаны несмотря на то что слова разные
    
    #return float((100-fuzz.ratio(a,b))/100)

    
def distance(a,b):
    ### Put your code here
    ### В переменной synsets помимо нормализованного заголовка хранятся синсеты для каждого токена из заголовка.
    ### у синсетов есть гиперонимы про то как из брать здесь
    ### Если у токенов заголовков есть общие гиперонимы тогда заголовки связаны несмотря на то что слова разные
    
    a = set(a) 
    b = set(b) 
    f1score, overlap = f1measure(a, b)
    a = a - overlap
    b = b - overlap
    
    if len(a) == 0 or len(b) == 0:
        return 1.0 - f1score
    
    buff_a = {}
    buff_b = {}
    
    if lang == 'eng':
        buff_a = extract_hypornyms_en(a)
        buff_b = extract_hypornyms_en(b)
        
    overlap_hyp_cnt = 0
    
    #for word in a:
    #    for wordb in b:
    #        if len(buff_a[word].intersection(buff_b[wordb])) > 0:
    #            overlap_hyp_cnt += 1
    
    overlap = a.intersection(b)
    overlap_hyp_cnt = len(overlap) 
    
    recl_hyp = overlap_hyp_cnt/len(a)
    prec_hyp = overlap_hyp_cnt/len(b)
    f1score_hyp = 2*recl_hyp/(recl_hyp + prec_hyp) if overlap_hyp_cnt > 0 else 0
    f1res = (2*f1score+f1score_hyp)/3
    return (1.0 - f1res)

buff = list(res.items())
dist = np.zeros((len(buff),len(buff)))
for lli,ll in enumerate(buff):
    for rri,rr in enumerate(buff):
        dist[lli,rri] = distance(ll[0], rr[0])
        

In [96]:
dist

array([[0.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 0.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 0.80952381, 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 0.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 0.        , 0.75757576,
        1.        , 1.        , 0.85964912, 1.        , 1.        ,
        1.        , 1.        , 1.        , 1

In [101]:
buff[0][0]

('much', 'card', 'deck', 'well', 'shuffled')

#### Top ten closest articles with fuzzy metrics of titles

In [97]:
@interact(ind=(0,len(buff)-1,1))
def h(ind=0):
    pp = pprint.PrettyPrinter(indent=4)
    print(' '.join(buff[ind][0]))
    pp.pprint([buff[i][0] for i in dist[ind][:].argsort()[1:11]])


interactive(children=(IntSlider(value=0, description='ind', max=19), Output()), _dom_classes=('widget-interact…

In [98]:
@interact(ind=(0,len(buff)-1,1))
def hypernyms(ind=0):
    pp = pprint.PrettyPrinter(indent=4)
    print(' '.join(buff[ind][0]))
    pp.pprint(buff[ind][1])

interactive(children=(IntSlider(value=0, description='ind', max=19), Output()), _dom_classes=('widget-interact…